###Кредитный скоринг
####При принятии решения о выдаче кредита или займа учитывается т.н. «Кредитный скоринг» — рейтинг платежеспособности клиента. ИИ на основе модели, которую просчитывает машинное обучение — в ней много параметров — возраст, зарплата, кредитная история, наличие недвижимости, автомобиля, судимости и других признаков, после обработки которых выносится положительное или отрицательное решение

# Данные:
[скачать](https://drive.google.com/file/d/1MuAyZiIm3b_r-AgQSj78tsRPqZpvv_2s/view?usp=sharing)

**application_record.csv**
*   Feature name	Explanation	Remarks
*   ID	Client number	
*   CODE_GENDER	Gender	
*   FLAG_OWN_CAR	Is there a car	
*   FLAG_OWN_REALTY	Is there a property	
*   CNT_CHILDREN	Number of children	
*   AMT_INCOME_TOTAL	Annual income	
*   NAME_INCOME_TYPE	Income category	
*   NAME_EDUCATION_TYPE	Education level	
*   NAME_FAMILY_STATUS	Marital status	
*   NAME_HOUSING_TYPE	Way of living	
*   DAYS_BIRTH	Birthday	Count backwards from current day (0), -1 means yesterday
*   DAYS_EMPLOYED	Start date of employment	Count backwards from current day(0). If positive, it means the person currently unemployed.
FLAG_MOBIL	Is there a mobile phone	
*   FLAG_WORK_PHONE	Is there a work phone	
*   FLAG_PHONE	Is there a phone	
*   FLAG_EMAIL	Is there an email	
*   OCCUPATION_TYPE	Occupation	
*   CNT_FAM_MEMBERS	Family size	

**credit_record.csv**
*   Feature name	Explanation	Remarks
*   ID	Client number	
*   MONTHS_BALANCE	Record month	The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on
*   STATUS	Status	
   *   0: 1-29 days past due
   *   1: 30-59 days past due 
   *   2: 60-89 days overdue 
   *   3: 90-119 days overdue 
   *   4: 120-149 days overdue 
    *   5: Overdue or bad debts, write-offs for more than 150 days
    *   C: paid off that month X: No loan for the month


In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=2f3c6bcb81ba05fabc6c4cc4bd752cc94ef5b703c51834ac858acf905f7ee37f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()


In [3]:
df = spark.read.csv('application_record.csv', header = True)
record = spark.read.csv('credit_record.csv', header = True)

In [4]:
df.printSchema(), record.printSchema()

root
 |-- ID: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: string (nullable = true)
 |-- AMT_INCOME_TOTAL: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- DAYS_BIRTH: string (nullable = true)
 |-- DAYS_EMPLOYED: string (nullable = true)
 |-- FLAG_MOBIL: string (nullable = true)
 |-- FLAG_WORK_PHONE: string (nullable = true)
 |-- FLAG_PHONE: string (nullable = true)
 |-- FLAG_EMAIL: string (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- CNT_FAM_MEMBERS: string (nullable = true)

root
 |-- ID: string (nullable = true)
 |-- MONTHS_BALANCE: string (nullable = true)
 |-- STATUS: string (nullable = true)



(None, None)

In [5]:
df.show(5)

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|         1

In [6]:
df.describe().show()

+-------+-----------------+-----------+------------+---------------+-------------------+------------------+--------------------+--------------------+------------------+-----------------+-------------------+-----------------+----------+------------------+-------------------+-------------------+--------------------+-----------------+
|summary|               ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|       CNT_CHILDREN|  AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|         DAYS_BIRTH|    DAYS_EMPLOYED|FLAG_MOBIL|   FLAG_WORK_PHONE|         FLAG_PHONE|         FLAG_EMAIL|     OCCUPATION_TYPE|  CNT_FAM_MEMBERS|
+-------+-----------------+-----------+------------+---------------+-------------------+------------------+--------------------+--------------------+------------------+-----------------+-------------------+-----------------+----------+------------------+-------------------+-------------------+--------------------+-----------------

In [7]:
from pyspark.sql.types import IntegerType
record = record \
  .withColumn("MONTHS_BALANCE" ,
              record["MONTHS_BALANCE"]
              .cast(IntegerType()))

In [8]:
new_record = record.groupBy('ID').agg(F.min('MONTHS_BALANCE')*-1) 
new_df = df.join(new_record,on=['ID'])
new_df.sort('ID').show(5)

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+--------------------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|(min(MONTHS_BALANCE) * -1)|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+--------------------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher edu

In [9]:
record = record.withColumn('dep_value', F.when((F.col('STATUS') == '2') | (F.col('STATUS') == '3') | (F.col('STATUS') == '4') | (F.col('STATUS') == '5'), 'Yes') 
                          .otherwise(None)
                           )

In [10]:
cpunt = record.groupby('ID', 'dep_value').count().sort('ID')

In [11]:
cpunt.describe().show()

+-------+-----------------+---------+------------------+
|summary|               ID|dep_value|             count|
+-------+-----------------+---------+------------------+
|  count|            46652|      667|             46652|
|   mean|5070292.559140015|     null|22.476528337477493|
| stddev|45428.67009852297|     null|15.515730399549481|
|    min|          5001711|      Yes|                 1|
|    max|          5150487|      Yes|                61|
+-------+-----------------+---------+------------------+



In [12]:
cpunt = cpunt.drop('count')
new_df = new_df.join(cpunt,on=['ID'], how='inner')
new_df = new_df.withColumn('target', F.when(F.col('dep_value') == 'Yes', 1).otherwise(0))

In [13]:
new_df.show(1, vertical=True)

-RECORD 0------------------------------------------
 ID                         | 5009208              
 CODE_GENDER                | F                    
 FLAG_OWN_CAR               | N                    
 FLAG_OWN_REALTY            | N                    
 CNT_CHILDREN               | 0                    
 AMT_INCOME_TOTAL           | 292500.0             
 NAME_INCOME_TYPE           | State servant        
 NAME_EDUCATION_TYPE        | Secondary / secon... 
 NAME_FAMILY_STATUS         | Married              
 NAME_HOUSING_TYPE          | House / apartment    
 DAYS_BIRTH                 | -20953               
 DAYS_EMPLOYED              | -8684                
 FLAG_MOBIL                 | 1                    
 FLAG_WORK_PHONE            | 1                    
 FLAG_PHONE                 | 1                    
 FLAG_EMAIL                 | 0                    
 OCCUPATION_TYPE            | Accountants          
 CNT_FAM_MEMBERS            | 2.0                  
 (min(MONTHS

In [14]:
features = ['AMT_INCOME_TOTAL', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN']	
target = ['target']
dataset = new_df[features + target]

In [15]:
dataset.describe().show()

+-------+------------------+-----------+------------+---------------+-------------------+-------------------+
|summary|  AMT_INCOME_TOTAL|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|       CNT_CHILDREN|             target|
+-------+------------------+-----------+------------+---------------+-------------------+-------------------+
|  count|             37073|      37073|       37073|          37073|              37073|              37073|
|   mean|186672.09548728185|       null|        null|           null|0.43028619210746366|0.01661586599411971|
| stddev|101718.37229069635|       null|        null|           null| 0.7423555652578205|0.12782886899813548|
|    min|          101250.0|          F|           N|              N|                  0|                  0|
|    max|           99900.0|          M|           Y|              Y|                  7|                  1|
+-------+------------------+-----------+------------+---------------+-------------------+-------------------+



In [16]:
dataset.printSchema()

root
 |-- AMT_INCOME_TOTAL: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: string (nullable = true)
 |-- target: integer (nullable = false)



In [17]:
dataset = dataset \
  .withColumn("AMT_INCOME_TOTAL" ,
              dataset["AMT_INCOME_TOTAL"]
              .cast(IntegerType()))\
  .withColumn("CNT_CHILDREN" ,
              dataset["CNT_CHILDREN"]
              .cast(IntegerType()))

In [18]:
X_train, X_test = dataset.randomSplit([0.7, 0.3], seed=42)

In [19]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler

indexer = StringIndexer(inputCols = ["CODE_GENDER","FLAG_OWN_CAR", "FLAG_OWN_REALTY"], 
                        outputCols = ["CODE_GENDER_Index","FLAG_OWN_CAR_Index", "FLAG_OWN_REALTY_Index"])

ohe = OneHotEncoder(inputCols = ["CODE_GENDER_Index","FLAG_OWN_CAR_Index", "FLAG_OWN_REALTY_Index"],
                    outputCols = ["CODE_GENDER_ohe","FLAG_OWN_CAR_ohe", "FLAG_OWN_REALTY_ohe"])

vector_assembler_num = VectorAssembler(inputCols = ["AMT_INCOME_TOTAL", "CNT_CHILDREN"],
                                       outputCol = "features_num")
mms = MinMaxScaler(inputCol = "features_num",
                    outputCol = "features_num_mms")

vector_assembler = VectorAssembler(inputCols = ["AMT_INCOME_TOTAL", "CODE_GENDER_ohe","FLAG_OWN_CAR_ohe", "FLAG_OWN_REALTY_ohe", "CNT_CHILDREN"], 
                                   outputCol = "features")

In [20]:
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features', labelCol='target')

imputer = [indexer, ohe, vector_assembler_num, mms, vector_assembler, lr]

pipe = Pipeline(stages = imputer)

In [21]:
model = pipe.fit(X_train)

In [22]:
train_pred = model.transform(X_train)
test_pred = model.transform(X_test)

In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluatorMulti = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction")
train_acc = evaluatorMulti.evaluate(train_pred, {evaluatorMulti.metricName: "accuracy"})
test_acc = evaluatorMulti.evaluate(test_pred, {evaluatorMulti.metricName: "accuracy"})
print(f'Точность модели в тренировочной выборке равна {train_acc}, в тестовой выборке - {test_acc}')

Точность модели в тренировочной выборке равна 0.9832097437276675, в тестовой выборке - 0.9837950389281188
